In [40]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
%load_ext jupyter_black

In [12]:
voters_df=pd.read_csv('~/Desktop/voters.csv',index_col=[0])
voters_df.fillna(' ',inplace=True)

In [ ]:
voters_df

In [26]:
voters_df.sort_values(by=['ward','precinct','fullstname'],ascending=True,inplace=True)


addresses_df=voters_df[['ward','precinct','addressid','wp_address_id']].groupby(['ward','precinct','addressid']).first().reset_index()
addresses_df['form_id']=addresses_df[['ward','precinct','addressid','wp_address_id']].groupby(['ward','precinct']).cumcount()
addresses_df['form_id']=[f'{w}{p}{f}' for w,p,f in zip(addresses_df.ward,addresses_df.precinct,addresses_df.form_id)]
addresses_df
voters_df=voters_df.merge(addresses_df[['form_id','addressid']],on='addressid')
voters_df.to_csv('~/Desktop/voters_with_formid.csv')

In [27]:
df=pd.read_csv('~/Desktop/selected_voters_1_1.csv')
df.fillna(' ',inplace=True)


In [47]:
config = {
    "write_out_cols": [
        "form_id",
        "address",
        "apt",
        "fullname",
        "age_on_election_day",
        "occupation",
    ],
    "groupby_cols": ["form_id", "address", "apt", "fullname"],
    "roads": {
        "6A1": ["BURR RD", "ELLISON RD", "FENNO RD", "MANAMET RD"],
        "6B1": ["MORELAND AVE", "ALBION PL", "NEWBURY ST", "KENMORE ST"],
    },
}
from utils.io import dict_to_yaml, yaml_to_dict

dict_to_yaml(config, "canvass.yml")

In [48]:
d = yaml_to_dict("/Users/lindseygulden/dev/newton/voters/canvass.yml")

In [56]:
roadlist = ["NEHOLDEN RD", "FISHER AVE", {"WALNUT ST": [960, 1080]}]

r = [list(x.values())[0] if isinstance(x, dict) else x for x in roadlist]
r

['NEHOLDEN RD', 'FISHER AVE', [960, 1080]]

In [ ]:
for x in roadlist:
    if isinstance(x,dict):
        streetname=list(x.keys())[0]
        numbers=list(x.values())[0]
        out_list.append(out_df.loc[(out_df.fullstname==streetname) & (out_df.number>=numbers[0] & out_df.number<=numbers[1])])
    else:
        out_list.append(out_df.loc[(out_df.fullstname==x)])

{'voter_file': '/Users/lindseygulden/Desktop/voters_with_formid.csv',
 'output_dir': '/Users/lindseygulden/Desktop/canvass_lists',
 'list_generation_id': 'canvass_set_1',
 'groupby_cols': ['form_id', 'address', 'apt', 'fullname'],
 'street_col': 'fullstname',
 'roads': {'6A1': ['BURR RD', 'ELLISON RD', 'FENNO RD', 'MANEMET RD'],
  '6B1': ['KENMORE ST',
   'PINE CREST RD',
   'MORELAND AVE',
   'ALBION PL',
   'NEWBURY ST'],
  '6B2': ['KINGMAN RD', 'FISHER AVE', {'WALNUT ST': [960, 1080]}],
  '6B3': ['SAXON RD', 'LAKEWOOD RD', 'NORMAN RD'],
  '6B4': ['HYDE AVE']},
 'write_out_cols': ['form_id',
  'address',
  'apt',
  'fullname',
  'age_on_election_day',
  'occupation']}

In [58]:
sorted(list(voters_df.fullstname.unique()))

['ABBOTT ST',
 'ABERDEEN ST',
 'ACACIA AVE',
 'ACADEMY RD',
 'ACORN DR',
 'ADAMS AVE',
 'ADAMS CT',
 'ADAMS ST',
 'ADAMS TER',
 'ADELINE RD',
 'ADELLA AVE',
 'ADENA RD',
 'AGAWAM RD',
 'ALBA CIR',
 'ALBAN RD',
 'ALBEMARLE RD',
 'ALBERT RD',
 'ALBION PL',
 'ALBION ST',
 'ALDEN PL',
 'ALDEN ST',
 'ALDERWOOD RD',
 'ALEXANDER RD',
 'ALGONQUIN RD',
 'ALLEN AVE',
 'ALLERTON RD',
 'ALLISON ST',
 'ALLSTON ST',
 'AMHERST RD',
 'AMY CIR',
 'ANDREW ST',
 'ANGIER CIR',
 'ANITA CIR',
 'ANNAPOLIS RD',
 'ANNAWAN RD',
 'ANTHONY CIR',
 'ANTHONY RD',
 'ANTONELLIS CIR',
 'APPLEGARTH ST',
 'APPLETON CIR',
 'ARAPAHOE RD',
 'ARBOR RD',
 'ARDEN RD',
 'ARDMORE RD',
 'ARDMORE TER',
 'ARLINGTON ST',
 'ARLO RD',
 'ARNOLD RD',
 'ARUNDEL TER',
 'ASCENTA TER',
 'ASH ST',
 'ASHCROFT RD',
 'ASHEVILLE RD',
 'ASHFORD RD',
 'ASHMONT AVE',
 'ASHMONT RD',
 'ASHTON AVE',
 'ASPEN AVE',
 'ATHELSTANE RD',
 'ATKINSON ST',
 'ATWOOD AVE',
 'AUBURN ST',
 'AUBURN TER',
 'AUBURNDALE AVE',
 'AUDUBON DR',
 'AUSTIN ST',
 'AVALON RD',


In [57]:
voters_df.loc[voters_df.fullstname == "NEHOLDEN RD"]

,addressid,wp_address_id,voter_id_number,fullname,first_name,number,address,fullstname,apt,party_affiliation,...,finance,muckity_muck,helping,multifamily,men_over_60,women,gender,lat,lon,form_id


In [44]:
for name, roadlist in config["roads"].items():
    print(
        df[config["write_out_cols"]]
        .loc[df.fullstname.isin(roadlist)]
        .groupby(config["groupby_cols"])
        .first()
    )

Empty DataFrame
Columns: [age_on_election_day, occupation]
Index: []
                                                   age_on_election_day  \
form_id address         apt fullname                                     
640     3 ALBION PL         JENNA CASTELLOT                         35   
                            JOHN CASTELLOT                          73   
                            KATHRYN BLUM                            74   
                            MICHAEL BLUM                            72   
                            VIRGINIA MERLINI                        73   
6321    12 ALBION PL        MARCIA ROSENBAUM                        67   
6322    16 ALBION PL        JARED KESSELHEIM                        48   
6323    2 ALBION PL         ADELE KAUFFMAN                          72   
                            BARRY COHEN                             73   
6325    25 ALBION PL        BARBARA ATWOOD                          84   
                            EBEN ATWOOD    

In [46]:
sorted(voters_df.fullstname.unique())

['ABBOTT ST',
 'ABERDEEN ST',
 'ACACIA AVE',
 'ACADEMY RD',
 'ACORN DR',
 'ADAMS AVE',
 'ADAMS CT',
 'ADAMS ST',
 'ADAMS TER',
 'ADELINE RD',
 'ADELLA AVE',
 'ADENA RD',
 'AGAWAM RD',
 'ALBA CIR',
 'ALBAN RD',
 'ALBEMARLE RD',
 'ALBERT RD',
 'ALBION PL',
 'ALBION ST',
 'ALDEN PL',
 'ALDEN ST',
 'ALDERWOOD RD',
 'ALEXANDER RD',
 'ALGONQUIN RD',
 'ALLEN AVE',
 'ALLERTON RD',
 'ALLISON ST',
 'ALLSTON ST',
 'AMHERST RD',
 'AMY CIR',
 'ANDREW ST',
 'ANGIER CIR',
 'ANITA CIR',
 'ANNAPOLIS RD',
 'ANNAWAN RD',
 'ANTHONY CIR',
 'ANTHONY RD',
 'ANTONELLIS CIR',
 'APPLEGARTH ST',
 'APPLETON CIR',
 'ARAPAHOE RD',
 'ARBOR RD',
 'ARDEN RD',
 'ARDMORE RD',
 'ARDMORE TER',
 'ARLINGTON ST',
 'ARLO RD',
 'ARNOLD RD',
 'ARUNDEL TER',
 'ASCENTA TER',
 'ASH ST',
 'ASHCROFT RD',
 'ASHEVILLE RD',
 'ASHFORD RD',
 'ASHMONT AVE',
 'ASHMONT RD',
 'ASHTON AVE',
 'ASPEN AVE',
 'ATHELSTANE RD',
 'ATKINSON ST',
 'ATWOOD AVE',
 'AUBURN ST',
 'AUBURN TER',
 'AUBURNDALE AVE',
 'AUDUBON DR',
 'AUSTIN ST',
 'AVALON RD',
